In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import random
import wx
import wx.xrc
import os

# Global variables
plainfilePickerPath = ""
plainfileOsPath = ""
plainfilename = ""
plainnonExt_filename = ""
plainextFrom_filename = ""

keyfilePickerPath = ""
keyfileOsPath = ""
keyfilename = ""
keynonExt_filename = ""
keyextFrom_filename = ""




'''
Euclid's algorithm for determining the greatest common divisor
Use iteration to make it faster for larger integers
'''
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

'''
Euclid's extended algorithm for finding the multiplicative inverse of two numbers
'''
def multiplicative_inverse(a, b):
    """Returns a tuple (r, i, j) such that r = gcd(a, b) = ia + jb
    """
    # r = gcd(a,b) i = multiplicitive inverse of a mod b
    #      or      j = multiplicitive inverse of b mod a
    # Neg return values for i or j are made positive mod b or a respectively
    # Iterateive Version is faster and uses much less stack space
    x = 0
    y = 1
    lx = 1
    ly = 0
    oa = a  # Remember original a/b to remove
    ob = b  # negative values from return results
    while b != 0:
        q = a // b
        (a, b) = (b, a % b)
        (x, lx) = ((lx - (q * x)), x)
        (y, ly) = ((ly - (q * y)), y)
    if lx < 0:
        lx += ob  # If neg wrap modulo orignal b
    if ly < 0:
        ly += oa  # If neg wrap modulo orignal a
    # return a , lx, ly  # Return only positive values
    return lx

'''
Tests to see if a number is prime.
'''
def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2):
        if num % n == 0:
            return False
    return True

def generate_keypair(p, q, e):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')
    #n = pq
    n = p * q

    #Phi is the totient of n
    phi = (p-1) * (q-1)

    #Choose an integer e such that e and phi(n) are coprime
#     e = random.randrange(1, phi)

    #Use Euclid's Algorithm to verify that e and phi(n) are comprime
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)

    #Use Extended Euclid's Algorithm to generate the private key
    d = multiplicative_inverse(e, phi)
    
    #Return public and private keypair
    #Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))

def encrypt(pk, plaintext):
    #Unpack the key into it's components
    key, n = pk
    #Convert each letter in the plaintext to numbers based on the character using a^b mod m
    cipher = [(ord(char) ** key) % n for char in plaintext]
    #Return the array of bytes
    return cipher

def decrypt(pk, ciphertext):
    #Unpack the key into its components
    key, n = pk
    #Generate the plaintext based on the ciphertext and key using a^b mod m
    plain = [chr((char ** key) % n) for char in ciphertext]
    #Return the array of bytes as a string
    return ''.join(plain)


# In[3]:


class MainPanel ( wx.Frame ):
    
    def __init__( self, parent ):
        wx.Frame.__init__ ( self, parent, id = wx.ID_ANY, title = u"Tubes Kripto : RSA", pos = wx.DefaultPosition, size = wx.Size( 928,558 ), style = wx.DEFAULT_FRAME_STYLE|wx.TAB_TRAVERSAL )

        self.SetSizeHintsSz( wx.DefaultSize, wx.DefaultSize )
        self.SetBackgroundColour( wx.SystemSettings.GetColour( wx.SYS_COLOUR_INACTIVEBORDER ) )

        gbSizer1 = wx.GridBagSizer( 0, 0 )
        gbSizer1.SetFlexibleDirection( wx.BOTH )
        gbSizer1.SetNonFlexibleGrowMode( wx.FLEX_GROWMODE_SPECIFIED )

        self.m_staticText1 = wx.StaticText( self, wx.ID_ANY, u"p", wx.DefaultPosition, wx.DefaultSize, 0 )
        self.m_staticText1.Wrap( -1 )
        gbSizer1.Add( self.m_staticText1, wx.GBPosition( 1, 0 ), wx.GBSpan( 1, 1 ), wx.ALL, 5 )

        self.p_textCtrl = wx.TextCtrl( self, wx.ID_ANY, wx.EmptyString, wx.DefaultPosition, wx.DefaultSize, 0 )
        gbSizer1.Add( self.p_textCtrl, wx.GBPosition( 1, 1 ), wx.GBSpan( 1, 1 ), wx.ALL, 5 )

        self.m_staticText2 = wx.StaticText( self, wx.ID_ANY, u"q", wx.DefaultPosition, wx.DefaultSize, 0 )
        self.m_staticText2.Wrap( -1 )
        gbSizer1.Add( self.m_staticText2, wx.GBPosition( 1, 2 ), wx.GBSpan( 1, 1 ), wx.ALL, 5 )

        self.q_textCtrl = wx.TextCtrl( self, wx.ID_ANY, wx.EmptyString, wx.DefaultPosition, wx.DefaultSize, 0 )
        gbSizer1.Add( self.q_textCtrl, wx.GBPosition( 1, 3 ), wx.GBSpan( 1, 1 ), wx.ALL, 5 )

        self.m_staticText3 = wx.StaticText( self, wx.ID_ANY, u"e", wx.DefaultPosition, wx.DefaultSize, 0 )
        self.m_staticText3.Wrap( -1 )
        gbSizer1.Add( self.m_staticText3, wx.GBPosition( 1, 4 ), wx.GBSpan( 1, 1 ), wx.ALL, 5 )

        self.e_textCtrl = wx.TextCtrl( self, wx.ID_ANY, wx.EmptyString, wx.DefaultPosition, wx.DefaultSize, 0 )
        gbSizer1.Add( self.e_textCtrl, wx.GBPosition( 1, 5 ), wx.GBSpan( 1, 1 ), wx.ALL, 5 )

        self.genkey_button = wx.Button( self, wx.ID_ANY, u"Generate Key Files", wx.DefaultPosition, wx.DefaultSize, 0 )
        gbSizer1.Add( self.genkey_button, wx.GBPosition( 2, 0 ), wx.GBSpan( 1, 6 ), wx.ALIGN_CENTER|wx.ALL, 5 )

        self.m_staticline2 = wx.StaticLine( self, wx.ID_ANY, wx.DefaultPosition, wx.DefaultSize, wx.LI_HORIZONTAL )
        gbSizer1.Add( self.m_staticline2, wx.GBPosition( 3, 0 ), wx.GBSpan( 1, 6 ), wx.EXPAND |wx.ALL, 5 )

        self.m_staticText5 = wx.StaticText( self, wx.ID_ANY, u"Input Property :", wx.DefaultPosition, wx.DefaultSize, 0 )
        self.m_staticText5.Wrap( -1 )
        gbSizer1.Add( self.m_staticText5, wx.GBPosition( 0, 0 ), wx.GBSpan( 1, 6 ), wx.ALL|wx.EXPAND, 5 )

        self.m_staticText14 = wx.StaticText( self, wx.ID_ANY, u"Result : ", wx.DefaultPosition, wx.DefaultSize, 0 )
        self.m_staticText14.Wrap( -1 )
        self.m_staticText14.SetBackgroundColour( wx.SystemSettings.GetColour( wx.SYS_COLOUR_INFOBK ) )

        gbSizer1.Add( self.m_staticText14, wx.GBPosition( 0, 6 ), wx.GBSpan( 1, 1 ), wx.ALL, 5 )

        self.plainfile_filePicker = wx.FilePickerCtrl( self, wx.ID_ANY, wx.EmptyString, u"Select a file", u"*.*", wx.DefaultPosition, wx.DefaultSize, wx.FLP_DEFAULT_STYLE )
        gbSizer1.Add( self.plainfile_filePicker, wx.GBPosition( 5, 0 ), wx.GBSpan( 1, 6 ), wx.ALL|wx.EXPAND, 5 )

        self.encrypt_button = wx.Button( self, wx.ID_ANY, u"Encrypt", wx.DefaultPosition, wx.DefaultSize, 0 )
        gbSizer1.Add( self.encrypt_button, wx.GBPosition( 8, 0 ), wx.GBSpan( 1, 3 ), wx.ALIGN_LEFT|wx.ALL, 5 )

        self.decrypt_button = wx.Button( self, wx.ID_ANY, u"Decrypt", wx.DefaultPosition, wx.DefaultSize, 0 )
        gbSizer1.Add( self.decrypt_button, wx.GBPosition( 8, 3 ), wx.GBSpan( 1, 3 ), wx.ALIGN_RIGHT|wx.ALL, 5 )

        self.m_staticText16 = wx.StaticText( self, wx.ID_ANY, u"p : ", wx.DefaultPosition, wx.DefaultSize, 0 )
        self.m_staticText16.Wrap( -1 )
        gbSizer1.Add( self.m_staticText16, wx.GBPosition( 9, 1 ), wx.GBSpan( 1, 1 ), wx.ALIGN_RIGHT|wx.ALL, 5 )

        self.m_staticText17 = wx.StaticText( self, wx.ID_ANY, u"q : ", wx.DefaultPosition, wx.DefaultSize, 0 )
        self.m_staticText17.Wrap( -1 )
        gbSizer1.Add( self.m_staticText17, wx.GBPosition( 10, 1 ), wx.GBSpan( 1, 1 ), wx.ALIGN_RIGHT|wx.ALL, 5 )

        self.pRes_textCtrl = wx.TextCtrl( self, wx.ID_ANY, wx.EmptyString, wx.DefaultPosition, wx.DefaultSize, 0 )
        self.pRes_textCtrl.SetMinSize( wx.Size( 50,-1 ) )

        gbSizer1.Add( self.pRes_textCtrl, wx.GBPosition( 9, 3 ), wx.GBSpan( 1, 1 ), wx.ALL, 5 )

        self.qRes_textCtrl = wx.TextCtrl( self, wx.ID_ANY, wx.EmptyString, wx.DefaultPosition, wx.DefaultSize, 0 )
        self.qRes_textCtrl.SetMinSize( wx.Size( 50,-1 ) )

        gbSizer1.Add( self.qRes_textCtrl, wx.GBPosition( 10, 3 ), wx.GBSpan( 1, 1 ), wx.ALL, 5 )

        self.m_staticText22 = wx.StaticText( self, wx.ID_ANY, u"e : ", wx.DefaultPosition, wx.DefaultSize, 0 )
        self.m_staticText22.Wrap( -1 )
        gbSizer1.Add( self.m_staticText22, wx.GBPosition( 11, 1 ), wx.GBSpan( 1, 1 ), wx.ALIGN_RIGHT|wx.ALL, 5 )

        self.m_staticText23 = wx.StaticText( self, wx.ID_ANY, u"d : ", wx.DefaultPosition, wx.DefaultSize, 0 )
        self.m_staticText23.Wrap( -1 )
        gbSizer1.Add( self.m_staticText23, wx.GBPosition( 12, 1 ), wx.GBSpan( 1, 1 ), wx.ALIGN_RIGHT|wx.ALL, 5 )

        self.m_staticText24 = wx.StaticText( self, wx.ID_ANY, u"n : ", wx.DefaultPosition, wx.DefaultSize, 0 )
        self.m_staticText24.Wrap( -1 )
        gbSizer1.Add( self.m_staticText24, wx.GBPosition( 13, 1 ), wx.GBSpan( 1, 1 ), wx.ALIGN_RIGHT|wx.ALL, 5 )

        self.eRes_textCtrl = wx.TextCtrl( self, wx.ID_ANY, wx.EmptyString, wx.DefaultPosition, wx.DefaultSize, 0 )
        self.eRes_textCtrl.SetMinSize( wx.Size( 50,-1 ) )

        gbSizer1.Add( self.eRes_textCtrl, wx.GBPosition( 11, 3 ), wx.GBSpan( 1, 1 ), wx.ALL, 5 )

        self.dRes_textCtrl = wx.TextCtrl( self, wx.ID_ANY, wx.EmptyString, wx.DefaultPosition, wx.DefaultSize, 0 )
        self.dRes_textCtrl.SetMinSize( wx.Size( 50,-1 ) )

        gbSizer1.Add( self.dRes_textCtrl, wx.GBPosition( 12, 3 ), wx.GBSpan( 1, 1 ), wx.ALL, 5 )

        self.nRes_textCtrl = wx.TextCtrl( self, wx.ID_ANY, wx.EmptyString, wx.DefaultPosition, wx.DefaultSize, 0 )
        self.nRes_textCtrl.SetMinSize( wx.Size( 50,-1 ) )

        gbSizer1.Add( self.nRes_textCtrl, wx.GBPosition( 13, 3 ), wx.GBSpan( 1, 1 ), wx.ALL, 5 )

        self.m_staticText6 = wx.StaticText( self, wx.ID_ANY, u"Input Key File : ", wx.DefaultPosition, wx.DefaultSize, 0 )
        self.m_staticText6.Wrap( -1 )
        gbSizer1.Add( self.m_staticText6, wx.GBPosition( 6, 0 ), wx.GBSpan( 1, 3 ), wx.ALL, 5 )

        self.keyfile_filePicker = wx.FilePickerCtrl( self, wx.ID_ANY, wx.EmptyString, u"Select a file", u"*.*", wx.DefaultPosition, wx.DefaultSize, wx.FLP_DEFAULT_STYLE )
        gbSizer1.Add( self.keyfile_filePicker, wx.GBPosition( 7, 0 ), wx.GBSpan( 1, 6 ), wx.ALL|wx.EXPAND, 5 )

        self.m_staticText4 = wx.StaticText( self, wx.ID_ANY, u"Input Text File : ", wx.DefaultPosition, wx.DefaultSize, 0 )
        self.m_staticText4.Wrap( -1 )
        gbSizer1.Add( self.m_staticText4, wx.GBPosition( 4, 0 ), wx.GBSpan( 1, 3 ), wx.ALL|wx.EXPAND, 5 )

        self.m_panel1 = wx.Panel( self, wx.ID_ANY, wx.DefaultPosition, wx.DefaultSize, wx.TAB_TRAVERSAL )
        self.m_panel1.SetBackgroundColour( wx.SystemSettings.GetColour( wx.SYS_COLOUR_INFOBK ) )

        gbSizer2 = wx.GridBagSizer( 0, 0 )
        gbSizer2.SetFlexibleDirection( wx.BOTH )
        gbSizer2.SetNonFlexibleGrowMode( wx.FLEX_GROWMODE_SPECIFIED )

        self.m_staticText10 = wx.StaticText( self.m_panel1, wx.ID_ANY, u"Encrypted Text : ", wx.DefaultPosition, wx.DefaultSize, 0 )
        self.m_staticText10.Wrap( -1 )
        gbSizer2.Add( self.m_staticText10, wx.GBPosition( 0, 0 ), wx.GBSpan( 1, 50 ), wx.ALL, 5 )

        self.cipher_textCtrl = wx.TextCtrl( self.m_panel1, wx.ID_ANY, wx.EmptyString, wx.DefaultPosition, wx.DefaultSize, 0 )
        gbSizer2.Add( self.cipher_textCtrl, wx.GBPosition( 1, 0 ), wx.GBSpan( 1, 50 ), wx.ALL|wx.EXPAND, 5 )

        self.m_staticText12 = wx.StaticText( self.m_panel1, wx.ID_ANY, u"Decrypted Text : ", wx.DefaultPosition, wx.DefaultSize, 0 )
        self.m_staticText12.Wrap( -1 )
        gbSizer2.Add( self.m_staticText12, wx.GBPosition( 4, 0 ), wx.GBSpan( 1, 50 ), wx.ALL, 5 )

        self.m_textCtrl10 = wx.TextCtrl( self.m_panel1, wx.ID_ANY, wx.EmptyString, wx.DefaultPosition, wx.DefaultSize, 0 )
        gbSizer2.Add( self.m_textCtrl10, wx.GBPosition( 5, 0 ), wx.GBSpan( 1, 50 ), wx.ALL|wx.EXPAND, 5 )


        self.m_panel1.SetSizer( gbSizer2 )
        self.m_panel1.Layout()
        gbSizer2.Fit( self.m_panel1 )
        gbSizer1.Add( self.m_panel1, wx.GBPosition( 1, 6 ), wx.GBSpan( 8, 1 ), wx.EXPAND |wx.ALL, 5 )


        self.SetSizer( gbSizer1 )
        self.Layout()

        self.Centre( wx.BOTH )

        # Connect Events
        self.genkey_button.Bind( wx.EVT_BUTTON, self.generate_key )
        self.plainfile_filePicker.Bind( wx.EVT_FILEPICKER_CHANGED, self.plaintext_pathchanged )
        self.encrypt_button.Bind( wx.EVT_BUTTON, self.onclick_encrypt )
        self.decrypt_button.Bind( wx.EVT_BUTTON, self.onclick_decrypt )
        self.keyfile_filePicker.Bind( wx.EVT_FILEPICKER_CHANGED, self.key_pathchanged )

    def __del__( self ):
        pass


    # Virtual event handlers, overide them in your derived class
    def generate_key( self, event ):
        p = int(self.p_textCtrl.GetValue())
        q = int(self.q_textCtrl.GetValue())
        e = int(self.e_textCtrl.GetValue())
        
        public, private = generate_keypair(p, q, e)

        kpub = open("public_key.pub","w+")
        public_key = public
        kpub.write(str(public_key))
        kpub = open("public_key.pub","w+")
        public_key = public
        kpub.write(str(public_key))
        print(public_key)

        

        ####################################################

        kpri = open("private_key.pri","w+")
        private_key = private
        kpri.write(str(private_key))
        kpri = open("private_key.pri","w+")
        private_key = private
        kpri.write(str(private_key))
        print(private_key)

        key_pri = open("private_key.pri", "r")
        pri_key = key_pri.read()
        pri_key = pri_key.lstrip('(')
        pri_key = pri_key.rstrip(')')
        print("kunci privat = ", key_pri.read())
        pri_key = pri_key.split(',')
        pri_key = (int(pri_key[0]),int(pri_key[1]))
        print(pri_key)
        
        self.pRes_textCtrl.SetValue(str(p))
        self.qRes_textCtrl.SetValue(str(q))
        self.eRes_textCtrl.SetValue(str(e))
        self.dRes_textCtrl.SetValue(str(pri_key[0]))
        self.nRes_textCtrl.SetValue(str(pri_key[1]))
        
        event.Skip()
    
    def onclick_encrypt( self, event ):
        global plainfilename
        global keyfilename
        plainfile = open(plainfilename,'r')
        message = plainfile.read()
        key_pub = open(keyfilename, "r")
        pub_key = key_pub.read()
        pub_key = pub_key.lstrip('(')
        pub_key = pub_key.rstrip(')')
        print("kunci publik = ", key_pub.read())
        pub_key = pub_key.split(',')
        pub_key = (int(pub_key[0]),int(pub_key[1]))
        print(pub_key)
        
        
        encrypted_msg = encrypt(pub_key, message)
        encrypted_msg = ''.join(map(lambda x: str(x), encrypted_msg))
        self.cipher_textCtrl.SetValue(str(encrypted_msg))
        f_en = open("encrypted_msg.txt","w+")
        f_en.write(encrypted_msg)
        event.Skip()

    def onclick_decrypt( self, event ):
        global plainfilename
        global keyfilename
        cipher = open(plainfilename,'r')
        message = cipher.read()
        
        key_pri = open(keyfilename, "r")
        pri_key = key_pri.read()
        pri_key = pri_key.lstrip('(')
        pri_key = pri_key.rstrip(')')
        print("kunci privat = ", key_pri.read())
        pri_key = pri_key.split(',')
        pri_key = (int(pri_key[0]),int(pri_key[1]))

        decrypted_msg = decrypt(pri_key, message)
        self.m_textCtrl10.SetValue(str(decrypted_msg))
        event.Skip()
        
    def plaintext_pathchanged( self, event ):
        global plainfilePickerPath
        global plainfilename
        global plainnonExt_filename
        global plainextFrom_filename
        plainfilePickerPath = self.plainfile_filePicker.GetPath()
#         filepath = filepickerpath.replace("\\","//")
        filepath = os.path.normpath(plainfilePickerPath)
        head, tail = os.path.split(filepath)
        plainfilename = tail
        plainnonExt_filename = plainfilename.split(".")[0]
        plainextFrom_filename = plainfilename.split(".")[1].lower()
        print("nonExt_filename : " + plainnonExt_filename)
        print("filepath : " + filepath)
        print("filename : " + plainfilename)
        
        event.Skip()
        
    def key_pathchanged( self, event ):
        global keyfilePickerPath
        global keyfilename
        global keynonExt_filename
        global keyextFrom_filename
        keyfilePickerPath = self.keyfile_filePicker.GetPath()
#         filepath = filepickerpath.replace("\\","//")
        filepath = os.path.normpath(keyfilePickerPath)
        head, tail = os.path.split(filepath)
        keyfilename = tail
        keynonExt_filename = keyfilename.split(".")[0]
        keyextFrom_filename = keyfilename.split(".")[1].lower()
        print("nonExt_filename : " + keynonExt_filename)
        print("filepath : " + filepath)
        print("filename : " + keyfilename)
        
        event.Skip()
if __name__ == '__main__':


    app = wx.PySimpleApp()
    frame = MainPanel(None)
    frame.Show()
    app.MainLoop()





c:\users\acer\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:407: wxPyDeprecationWarning: Using deprecated class PySimpleApp. Use :class:`App` instead.
c:\users\acer\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:130: wxPyDeprecationWarning: Call to deprecated item. Use SetSizeHints instead.


(13, 3337)
(2477, 3337)
kunci privat =  
(2477, 3337)
nonExt_filename : test
filepath : D:\Study\Kriptografi\test.txt
filename : test.txt
nonExt_filename : public_key
filepath : D:\Study\Kriptografi\public_key.pub
filename : public_key.pub
kunci publik =  
(13, 3337)
nonExt_filename : private_key
filepath : D:\Study\Kriptografi\private_key.pri
filename : private_key.pri
nonExt_filename : encrypted_msg
filepath : D:\Study\Kriptografi\encrypted_msg.txt
filename : encrypted_msg.txt
nonExt_filename : private_key
filepath : D:\Study\Kriptografi\private_key.pri
filename : private_key.pri
kunci privat =  
(2477, 3337)


TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'